# Universidad del Valle de Guatemala
# (CC3092) Security Data Science
# Laboratorio 4 - Análisis de Malware

Miembros del equipo de trabajo:
- Yongbum Park (20117)
- Santiago Taracena Puga (20017)

## Parte 1

### Creación del dataset

Para llevar a cabo la creación del dataset, se procedió con un análisis estático de los archivos de malware proporcionados utilizando la herramienta pefile. Esta herramienta permitió examinar en profundidad los archivos ejecutables y extraer información relevante, como el PE header, el nombre de las secciones y las llamadas a funciones realizadas.

Durante el análisis, se observó que los archivos de malware presentaban una variedad de características y comportamientos, lo que subraya la importancia de un enfoque detallado y exhaustivo en la extracción de características. Se identificaron diferentes familias de malware entre los ejemplos proporcionados, lo que respalda la necesidad de una clasificación precisa para desarrollar respuestas y controles adecuados.

Posteriormente, se procedió con la exploración y el preprocesamiento de los datos obtenidos del análisis estático. Se identificaron columnas que requerían técnicas de preprocesamiento para garantizar la calidad y la manipulación adecuada de la información. Entre las técnicas de preprocesamiento aplicadas se incluyeron la normalización de datos, el manejo de valores faltantes y la codificación de variables categóricas, con el fin de optimizar la preparación de los datos para su posterior análisis mediante algoritmos de aprendizaje no supervisado.

En conclusión, la etapa de creación del dataset proporcionó una base sólida para el análisis y la clasificación de familias de malware. El análisis estático de los archivos de malware, junto con el preprocesamiento adecuado de los datos, constituyen pasos fundamentales para avanzar hacia la implementación de algoritmos de clasificación y la determinación de similitudes entre las familias de malware identificadas.

La primera acción que resulta necesaria al momento de comenzar con la creación del dataset a utilizar durante el laboratorio consiste en el proceso de importar las librerías necesarias para abrir el malware en Python y analizarlo correctamente. Las librerías que necesitamos en este caso en particular son `os`, `hashlib`, `pefile` y `pandas` para guardar el dataset en el formato .csv que necesitamos.

In [1]:
# Librerías necesarias para armar el dataset.
import os
import hashlib
import pefile
import pandas as pd

Con las librerías necesarias disponibles para su utilización, necesitamos conseguir los hashes de cada archivo para poder utilizarlos con el objetivo de identificar cada malware del cual se están extrayendo características dentro del dataset. Para esto hemos desarrollado una función `get_file_hashes`, que obtiene los hashes utilizando la librería `hashlib` y los retorna para poder colocarlos como datos de columnas dentro de nuestro dataset.

In [2]:
# Función get_file_hashes, que obtiene los hashes de cada archivo de malware.
def get_file_hashes(file_path):

    # Apertura del archivo.
    with open(file_path, "rb") as file:

        # Obtención del contenido y los hashes del archivo.
        content = file.read()
        md5_hash = hashlib.md5(content).hexdigest()
        sha1_hash = hashlib.sha1(content).hexdigest()
        sha256_hash = hashlib.sha256(content).hexdigest()

    # Retorno de los hashes del archivo.
    return md5_hash, sha1_hash, sha256_hash

Con la función que extrae los hashes finalizada, es necesario crear otra función que nos brinde todas las características del malware que necesitamos utilizar con el objetivo de identificar los clústers necesarios. La función es bastante extensa, pero se encuentra propiamente comentada en cada una de sus secciones con el objetivo de facilitar su entendimiento. Utiliza principalmente la librería `pefile`, y la misma también se llama `extract_pe_features`, y finalmente tiene como objetivo, valga la redundancia, extraer las características de cada archivo de malware.

In [3]:
# Función extract_pe_features, que extrae las características necesarias del malware.
def extract_pe_features(file_path):

    # Lista de features que se retornarán.
    pe_features = []

    # Bloque try-catch que abre y accede al malware.
    try:

        # Primeras catacterísticas a extraer.
        pe = pefile.PE(file_path)
        pe_features.append(len(pe.sections))  # Número de secciones.
        pe_features.append(pe.FILE_HEADER.Machine)  # Arquitectura de la máquina.
        pe_features.append(pe.FILE_HEADER.Characteristics)  # Características del archivo.
        pe_features.append(pe.OPTIONAL_HEADER.SizeOfCode)  # Tamaño del código.
        pe_features.append(pe.OPTIONAL_HEADER.SizeOfInitializedData)  # Tamaño de datos inicializados.
        pe_features.append(pe.OPTIONAL_HEADER.SizeOfUninitializedData)  # Tamaño de datos no inicializados.
        pe_features.append(pe.OPTIONAL_HEADER.AddressOfEntryPoint)  # Dirección del punto de entrada.
        pe_features.append(pe.OPTIONAL_HEADER.ImageBase)  # Base de la imagen.
        pe_features.append(pe.OPTIONAL_HEADER.MajorSubsystemVersion)  # Versión del subsistema principal.
        pe_features.append(pe.OPTIONAL_HEADER.Subsystem)  # Subsistema.
        pe_features.append(pe.OPTIONAL_HEADER.DllCharacteristics)  # Características de DLL.
        
        # Checksum del archivo.
        pe_features.append(pe.OPTIONAL_HEADER.CheckSum)

        # Importaciones de funciones del archivo.
        imports = []
        for entry in pe.DIRECTORY_ENTRY_IMPORT:
            for imp in entry.imports:
                imports.append(imp.name.decode("utf-8", "ignore"))
        pe_features.append(imports)

        # Exportaciones de funciones del archivo.
        exports = []
        if (hasattr(pe, "DIRECTORY_ENTRY_EXPORT")):
            for exp in pe.DIRECTORY_ENTRY_EXPORT.symbols:
                exports.append(exp.name.decode("utf-8", "ignore"))
        pe_features.append(exports)

        # Tiempo de compilación del archivo.
        pe_features.append(pe.FILE_HEADER.TimeDateStamp)

        # Firma digital del archivo.
        pe_features.append("Signed" if (hasattr(pe, "DIRECTORY_ENTRY_CERT")) else "Unsigned")

        # Versiones de recursos del archivo.
        if (hasattr(pe, "VS_VERSIONINFO")):
            pe_features.append(pe.VS_VERSIONINFO.ProductVersion)
            pe_features.append(pe.VS_VERSIONINFO.FileVersion)
        else:
            pe_features.append(None)
            pe_features.append(None)

        # Entropía del archivo.
        section_entropies = [section.get_entropy() for section in pe.sections]
        pe_features.append(section_entropies)

        # Secciones ejecutables del archivo.
        executable_sections = [section.IMAGE_SCN_MEM_EXECUTE for section in pe.sections]
        pe_features.append(executable_sections)

        # Número de secciones ejecutables del archivo.
        executable_sections_length = sum(section.IMAGE_SCN_MEM_EXECUTE for section in pe.sections)
        pe_features.append(executable_sections_length)

        # TLS (Thread Local Storage) del archivo.
        pe_features.append(pe.OPTIONAL_HEADER.DATA_DIRECTORY[9].VirtualAddress)
        pe_features.append(pe.OPTIONAL_HEADER.DATA_DIRECTORY[9].Size)

        # Dimensiones del Entry Resource del archivo.
        if (hasattr(pe, "DIRECTORY_ENTRY_RESOURCE")):
            pe_features.append(len(pe.DIRECTORY_ENTRY_RESOURCE.entries))
            resource_size = sum(resource.directory.SizeOfData for resource in pe.DIRECTORY_ENTRY_RESOURCE.entries)
            pe_features.append(resource_size)
        else:
            pe_features.extend([0, 0])

    # Manejo de cualquier error manipulando el archivo.
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

    # Retorno de las features halladas.
    return pe_features

Con nuestra función de extracción de características de los archivos de malware proporcionados efectivamente completada, nos queda utilizarla junto con la función que fue creada para la obtención de hashes con el objetivo de crear el dataset. Debemos indicar la ruta del folder con el malware y la ruta del archivo .csv de salida con las características que necesitamos utilizar para el análisis.

In [4]:
# Rutas de entrada y de salida.
malware_folder = "./malware"
output_csv = "./data/malware-dataset.csv"

Con estas rutas configuradas correctamente, nos hace falta una lista con toda la data a almacenar con el objetivo de finalmente ser almacenada en el archivo .csv que también configuramos.

In [5]:
# Lista con los datos a guardar.
data = []

Todo está correctamente configurado, y ahora lo único que hace falta es codificar el proceso de iterar en la carpeta con el malware y extraer todas sus características, para finalmente guardarlas en la lista `data` que instanciamos anteriormente.

In [6]:
# Ciclo que itera el folder de malware y guarda las características adquiridas.
for file_name in os.listdir(malware_folder):
    file_path = os.path.join(malware_folder, file_name)
    md5_hash, sha1_hash, sha256_hash = get_file_hashes(file_path)
    pe_features = extract_pe_features(file_path)
    data.append([file_name, md5_hash, sha1_hash, sha256_hash] + pe_features)

Error processing ./malware/GFT4_7DDD3D72EAD03C7518F5D47650C8572: 'list' object has no attribute 'ProductVersion'
Error processing ./malware/EEE99EC8AA67B05407C01094184C33D2B5A44: 'ResourceDirData' object has no attribute 'SizeOfData'
Error processing ./malware/FGTR43_EF8E0FB20E7228C7492CCDC59D87C690: 'list' object has no attribute 'ProductVersion'
Error processing ./malware/QW2_4C6BDDCCA2695D6202DF38708E14FC7E: 'list' object has no attribute 'ProductVersion'


Error processing ./malware/JKK8CA6FE7A1315AF5AFEAC2961460A80569: 'ResourceDirData' object has no attribute 'SizeOfData'
Error processing ./malware/A316D5AECA269CA865077E7FFF356E7D: 'ResourceDirData' object has no attribute 'SizeOfData'
Error processing ./malware/.DS_Store: 'DOS Header magic not found.'
Error processing ./malware/65018CD542145A3792BA09985734C12A: 'ResourceDirData' object has no attribute 'SizeOfData'
Error processing ./malware/785003A405BC7A4EBCBB21DDB757BF3F: 'ResourceDirData' object has no attribute 'SizeOfData'
Error processing ./malware/JH78C0A33A1B472A8C16123FD696A5CE5EBB: 'ResourceDirData' object has no attribute 'SizeOfData'
Error processing ./malware/6FAA4740F99408D4D2DDDD0B09BBDEFD: 'ResourceDirData' object has no attribute 'SizeOfData'
Error processing ./malware/FTTR9EA3C16194CE354C244C1B74C46CD92E: 'ResourceDirData' object has no attribute 'SizeOfData'
Error processing ./malware/VBMM9_149B7BD7218AAB4E257D28469FDDB0D: 'list' object has no attribute 'ProductVer

Finalmente, estamos listos para escribir el archivo .csv con las características encontradas por parte de cada uno de los archivos. Debemos configurar las columnas, el dataframe, y finalmente llamar a la función `to_csv` para pasar el dataframe al formato .csv que necesitamos utilizar.

In [7]:
# Columnas del dataset.
columns = [
  "file_name", "md5", "sha1", "sha256", "num_sections", "machine", "characteristics",
  "size_of_code", "size_of_initialized_data", "size_of_uninitialized_data", "address_of_entry_point",
  "image_base", "major_subsystem_version", "subsystem", "dll_characteristics", "checksum",
  "imports", "exports", "time_date_stamp", "digital_signature", "product_version",
  "file_version", "section_entropies", "executable_sections", "executable_sections_length",
  "tls_virtual_address", "tls_size", "resources_section_entries", "resources_section_size",
]

# Instancia del dataframe con la data recolectada.
df = pd.DataFrame(data, columns=columns)

# Guardado del dataframe en formato .csv.
df.to_csv(output_csv, index=False)